In [97]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf 

In [30]:
cancellations_scotland_fp = os.path.join(os.getcwd(), "datasets", "cancellations_in_scotland.csv")
cancellations_scotland = pd.read_csv(cancellations_scotland_fp)

cancellations_hospital_fp = os.path.join(os.getcwd(), "datasets", "cancellations_by_hospital.csv")
cancellations_hospital = pd.read_csv(cancellations_hospital_fp)

cancellations_hb_fp = os.path.join(os.getcwd(), "datasets", "cancellations_by_hb.csv")
cancellations_hb = pd.read_csv(cancellations_hb_fp)

In [31]:
cancellations_scotland.isna().sum()

#print("number of rows", len(cancellations_scotland))

_id                              0
Month                            0
Country                          0
TotalOperations                  0
TotalOperationsQF              105
TotalCancelled                   0
TotalCancelledQF               105
CancelledByPatientReason         0
CancelledByPatientReasonQF     105
ClinicalReason                   0
ClinicalReasonQF               105
NonClinicalCapacityReason        0
NonClinicalCapacityReasonQF    105
OtherReason                      0
OtherReasonQF                  105
dtype: int64

In [32]:
cancellations_scotland

,_id,Month,Country,TotalOperations,TotalOperationsQF,TotalCancelled,TotalCancelledQF,CancelledByPatientReason,CancelledByPatientReasonQF,ClinicalReason,ClinicalReasonQF,NonClinicalCapacityReason,NonClinicalCapacityReasonQF,OtherReason,OtherReasonQF
0,1,201505,S92000003,28810,NaN,2695,NaN,1057,NaN,853,NaN,523,NaN,262,NaN
1,2,201506,S92000003,30941,NaN,2800,NaN,1194,NaN,870,NaN,449,NaN,287,NaN
2,3,201507,S92000003,27725,NaN,2642,NaN,1083,NaN,854,NaN,457,NaN,248,NaN
3,4,201508,S92000003,28948,NaN,2630,NaN,1080,NaN,912,NaN,491,NaN,147,NaN
4,5,201509,S92000003,30938,NaN,2829,NaN,1122,NaN,1000,NaN,535,NaN,172,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,101,202309,S92000003,23209,NaN,2105,NaN,739,NaN,738,NaN,541,NaN,87,NaN
101,102,202310,S92000003,24201,NaN,2286,NaN,775,NaN,797,NaN,580,NaN,134,NaN
102,103,202311,S92000003,25922,NaN,2277,NaN,737,NaN,820,NaN,582,NaN,138,NaN
103,104,202312,S92000003,20837,NaN,2014,NaN,677,NaN,725,NaN,515,NaN,97,NaN


In [33]:
cancellations_hb

,_id,Month,HBT,TotalOperations,TotalOperationsQF,TotalCancelled,TotalCancelledQF,CancelledByPatientReason,CancelledByPatientReasonQF,ClinicalReason,ClinicalReasonQF,NonClinicalCapacityReason,NonClinicalCapacityReasonQF,OtherReason,OtherReasonQF
0,1,201505,S08000015,1861,NaN,178,NaN,52,NaN,70,NaN,27,NaN,29,NaN
1,2,201505,S08000016,457,NaN,66,NaN,11,NaN,10,NaN,45,NaN,0,NaN
2,3,201505,S08000017,1433,NaN,113,NaN,33,NaN,56,NaN,15,NaN,9,NaN
3,4,201505,S08000019,1250,NaN,91,NaN,40,NaN,34,NaN,10,NaN,7,NaN
4,5,201505,S08000020,3310,NaN,264,NaN,110,NaN,71,NaN,76,NaN,7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,1571,202401,S08000029,1334,NaN,64,NaN,21,NaN,27,NaN,16,NaN,0,NaN
1571,1572,202401,S08000030,1550,NaN,131,NaN,46,NaN,47,NaN,5,NaN,33,NaN
1572,1573,202401,S08000031,5121,NaN,452,NaN,135,NaN,144,NaN,120,NaN,53,NaN
1573,1574,202401,S08000032,1930,NaN,167,NaN,93,NaN,43,NaN,27,NaN,4,NaN


In [34]:
cancellations_hb.isna().sum()

_id                               0
Month                             0
HBT                               1
TotalOperations                   0
TotalOperationsQF              1575
TotalCancelled                    0
TotalCancelledQF               1575
CancelledByPatientReason          0
CancelledByPatientReasonQF     1575
ClinicalReason                    0
ClinicalReasonQF               1575
NonClinicalCapacityReason         0
NonClinicalCapacityReasonQF    1575
OtherReason                       0
OtherReasonQF                  1575
dtype: int64

In [35]:
cancellations_hospital

,_id,Month,Hospital,TotalOperations,TotalOperationsQF,TotalCancelled,TotalCancelledQF,CancelledByPatientReason,CancelledByPatientReasonQF,ClinicalReason,ClinicalReasonQF,NonClinicalCapacityReason,NonClinicalCapacityReasonQF,OtherReason,OtherReasonQF
0,1,201505,A111H,884,NaN,67,NaN,21,NaN,25,NaN,16,NaN,5,NaN
1,2,201506,A111H,945,NaN,84,NaN,39,NaN,30,NaN,12,NaN,3,NaN
2,3,201507,A111H,948,NaN,72,NaN,25,NaN,33,NaN,6,NaN,8,NaN
3,4,201508,A111H,882,NaN,74,NaN,42,NaN,18,NaN,12,NaN,2,NaN
4,5,201509,A111H,845,NaN,67,NaN,32,NaN,30,NaN,3,NaN,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4497,4498,202401,Z102H,164,NaN,27,NaN,16,NaN,3,NaN,8,NaN,0,NaN
4498,4499,202401,T101H,822,NaN,69,NaN,26,NaN,27,NaN,2,NaN,14,NaN
4499,4500,202401,T202H,336,NaN,22,NaN,4,NaN,11,NaN,1,NaN,6,NaN
4500,4501,202401,T312H,392,NaN,40,NaN,16,NaN,9,NaN,2,NaN,13,NaN


In [36]:
cancellations_hospital.isna().sum()

_id                               0
Month                             0
Hospital                          0
TotalOperations                   0
TotalOperationsQF              4502
TotalCancelled                    0
TotalCancelledQF               4502
CancelledByPatientReason          0
CancelledByPatientReasonQF     4502
ClinicalReason                    0
ClinicalReasonQF               4502
NonClinicalCapacityReason         0
NonClinicalCapacityReasonQF    4502
OtherReason                       0
OtherReasonQF                  4502
dtype: int64

In [37]:
# Dropping columns where all the rows have Nan
cancellations_hospital.drop(columns=["TotalOperationsQF","TotalCancelledQF","CancelledByPatientReasonQF","ClinicalReasonQF","NonClinicalCapacityReasonQF","OtherReasonQF"], inplace=True)
cancellations_scotland.drop(columns=["TotalOperationsQF","TotalCancelledQF","CancelledByPatientReasonQF","ClinicalReasonQF","NonClinicalCapacityReasonQF","OtherReasonQF"], inplace=True)
cancellations_hb.drop(columns=["TotalOperationsQF","TotalCancelledQF","CancelledByPatientReasonQF","ClinicalReasonQF","NonClinicalCapacityReasonQF","OtherReasonQF"], inplace=True)


In [38]:
cancellations_hb.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1575 entries, 0 to 1574
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   _id                        1575 non-null   int64 
 1   Month                      1575 non-null   int64 
 2   HBT                        1574 non-null   object
 3   TotalOperations            1575 non-null   int64 
 4   TotalCancelled             1575 non-null   int64 
 5   CancelledByPatientReason   1575 non-null   int64 
 6   ClinicalReason             1575 non-null   int64 
 7   NonClinicalCapacityReason  1575 non-null   int64 
 8   OtherReason                1575 non-null   int64 
dtypes: int64(8), object(1)
memory usage: 110.9+ KB


In [78]:
cancellations_hb

,_id,Month,HBT,TotalOperations,TotalCancelled,CancelledByPatientReason,ClinicalReason,NonClinicalCapacityReason,OtherReason
0,1,201505,S08000015,1861,178,52,70,27,29
1,2,201505,S08000016,457,66,11,10,45,0
2,3,201505,S08000017,1433,113,33,56,15,9
3,4,201505,S08000019,1250,91,40,34,10,7
4,5,201505,S08000020,3310,264,110,71,76,7
...,...,...,...,...,...,...,...,...,...
1570,1571,202401,S08000029,1334,64,21,27,16,0
1571,1572,202401,S08000030,1550,131,46,47,5,33
1572,1573,202401,S08000031,5121,452,135,144,120,53
1573,1574,202401,S08000032,1930,167,93,43,27,4


In [88]:
cancellations_hb.HBT.value_counts()

#SB0801 is a special health board, Golden Jubilee University National Hospital

S08000015    105
S08000016    105
S08000017    105
S08000019    105
S08000020    105
S08000022    105
S08000024    105
S08000025    105
S08000026    105
S08000028    105
S08000029    105
S08000030    105
S08000031    105
S08000032    105
SB0801       104
Name: HBT, dtype: int64

In [89]:
(cancellations_hb.Month.value_counts() > 15).sum()
# exactly 15 health board for each month

0

In [64]:
cancellations_scotland

,_id,Month,Country,TotalOperations,TotalCancelled,CancelledByPatientReason,ClinicalReason,NonClinicalCapacityReason,OtherReason
0,1,201505,S92000003,28810,2695,1057,853,523,262
1,2,201506,S92000003,30941,2800,1194,870,449,287
2,3,201507,S92000003,27725,2642,1083,854,457,248
3,4,201508,S92000003,28948,2630,1080,912,491,147
4,5,201509,S92000003,30938,2829,1122,1000,535,172
...,...,...,...,...,...,...,...,...,...
100,101,202309,S92000003,23209,2105,739,738,541,87
101,102,202310,S92000003,24201,2286,775,797,580,134
102,103,202311,S92000003,25922,2277,737,820,582,138
103,104,202312,S92000003,20837,2014,677,725,515,97


In [65]:
# no duplication in the months
print((cancellations_scotland.Month.value_counts() > 1).sum())


0


In [66]:
cancellations_hospital

,_id,Month,Hospital,TotalOperations,TotalCancelled,CancelledByPatientReason,ClinicalReason,NonClinicalCapacityReason,OtherReason
0,1,201505,A111H,884,67,21,25,16,5
1,2,201506,A111H,945,84,39,30,12,3
2,3,201507,A111H,948,72,25,33,6,8
3,4,201508,A111H,882,74,42,18,12,2
4,5,201509,A111H,845,67,32,30,3,2
...,...,...,...,...,...,...,...,...,...
4497,4498,202401,Z102H,164,27,16,3,8,0
4498,4499,202401,T101H,822,69,26,27,2,14
4499,4500,202401,T202H,336,22,4,11,1,6
4500,4501,202401,T312H,392,40,16,9,2,13


In [103]:
cancellations_hospital.Hospital.value_counts()
#Only 49 hospitals, where there are 276 hospitals in Scotland, so a small data size

A111H    105
L106H    105
L308H    105
N101H    105
N102H    105
A210H    105
N161H    105
N411H    105
S116H    105
S308H    105
S314H    105
S316H    105
T101H    105
T202H    105
V217H    105
W107H    105
Z102H    105
L302H    105
N121H    105
D102H    105
G107H    105
H103H    105
G516H    105
G513H    105
G405H    105
B120H    105
C121H    105
C418H    105
H202H    105
F704H    105
H212H    104
C206H    103
G207H    103
C313H    103
F805H    102
T312H    102
Y144H    101
G306H    100
Y146H     74
V102H     73
S225H     71
Y177C     69
R103H     56
S113H     52
R101H     49
Y104H     35
S319H     35
F812H     10
H230H     10
Name: Hospital, dtype: int64

In [102]:
# pd.set_option('display.max_rows', None)
print((cancellations_hospital.Month.value_counts().to_string))
# exactly 15 health board for each month

<bound method Series.to_string of 202401    44
201805    44
201907    44
201906    44
201905    44
201712    44
201801    44
201802    44
201803    44
201804    44
201806    44
201904    44
201807    44
201808    44
201809    44
201810    44
201811    44
201812    44
201901    44
201902    44
201908    44
201903    44
202103    44
202312    44
202311    44
202310    44
202309    44
202308    44
202307    44
202306    44
202305    44
202304    44
202104    43
202105    43
202001    43
202002    43
202101    43
202012    43
202011    43
202010    43
202009    43
202008    43
202003    43
201506    43
201910    43
201911    43
201912    43
201505    43
201711    43
201602    43
201607    43
201606    43
201605    43
201604    43
201710    43
201603    43
201601    43
201609    43
201512    43
201511    43
201510    43
201509    43
201508    43
201507    43
201608    43
201909    43
201610    43
201706    43
201701    43
201703    43
201704    43
201705    43
201702    43
201707    43
2017